<a href="https://colab.research.google.com/github/joanizba/Spotifypred/blob/memoria/memoria_raul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Memoria Técnica – Entorno Big Data con Docker: Cassandra, Hadoop y Apache NiFi

## 🎯 Objetivo

El objetivo de este proyecto es crear un entorno de trabajo con tecnologías Big Data usando **contenedores Docker**, que incluya:

- **Apache Cassandra**: base de datos NoSQL distribuida.
- **Apache Hadoop**: sistema de archivos distribuido (HDFS).
- **Apache NiFi**: plataforma para automatización de flujos de datos.

Todos los contenedores están conectados entre sí mediante una red Docker personalizada.

---

## 🛠 Herramientas utilizadas

- **Docker**
- **Docker Compose**
- **Ubuntu (Host o WSL2)**
- **Google Chrome** (para interfaces web)
- **Google Colab** (para esta documentación)

---

## 📁 Estructura del Proyecto

```text
docker-cluster/
├── docker-compose.yml
└── README.md
```
## 🚀 Pasos para Crear el Entorno Big Data con Docker

### 1. Instalar Docker y Docker Compose

Lo primero es tener Docker y Docker Compose instalados en tu sistema para poder ejecutar los contenedores. Si ya tienes Docker instalado, puedes omitir esta parte.

**Para instalar Docker en Ubuntu:**
```text
sudo apt-get update
sudo apt-get install docker.io
sudo systemctl start docker
sudo systemctl enable docker
```
**Para instalar Docker Compose:**
```text
sudo apt-get install docker-compose
```

### 2. Configurar el Proyecto
Para que todo funcione, vamos a crear un archivo docker-compose.yml en un directorio de trabajo que especifique cómo crear los contenedores de Cassandra, Hadoop y NiFi.

**Crear el directorio del proyecto:**
```text
mkdir docker-cluster && cd docker-cluster
```
**Crear el archivo docker-compose.yml:**

Este archivo contiene la configuración de los tres contenedores (Cassandra, Hadoop y NiFi) y las redes para conectarlos.
```text
nano docker-compose.yml
```
Copia el siguiente contenido dentro de ese archivo.
``text
version: '3.9'

services:
  cassandra:
    image: cassandra:latest
    container_name: cassandra
    environment:
      - CASSANDRA_CLUSTER_NAME=TestCluster
      - CASSANDRA_START_RPC=true
    ports:
      - "9042:9042"
    networks:
      - my_network

  hadoop:
    image: bde2020/hadoop-namenode:latest
    container_name: hadoop
    environment:
      - CLUSTER_NAME=test
    ports:
      - "9870:9870"
      - "9000:9000"
    networks:
      - my_network
    depends_on:
      - cassandra

  nifi:
    image: apache/nifi:latest
    container_name: nifi
    ports:
      - "8080:8080"
    environment:
      - NIFI_WEB_HTTP_PORT=8080
    networks:
      - my_network
    depends_on:
      - cassandra
      - hadoop

networks:
  my_network:
    driver: bridge
``

### 3. Levantar los Contenedores
Una vez creado el archivo docker-compose.yml, puedes levantar todos los servicios utilizando el siguiente comando en la misma carpeta donde está el archivo:
```text
docker-compose up -d
```
Este comando descargará las imágenes de los contenedores y los ejecutará en segundo plano.

### 4. Verificar que los contenedores estén corriendo
Puedes verificar que todos los contenedores estén funcionando correctamente con:
```text
docker ps
```
Deberías ver los contenedores Cassandra, Hadoop y NiFi en ejecución.

### 5. Acceder a las interfaces web de los servicios
Hadoop: Accede a la interfaz de usuario de Hadoop a través de http://localhost:9870.

NiFi: Accede a la interfaz de usuario de NiFi a través de http://localhost:8080/nifi.

## 🧑‍💻 Conectar Cassandra con NiFi
Ahora vamos a probar la conectividad entre NiFi y Cassandra, para asegurarnos de que los dos servicios se pueden comunicar.

### Acceder al contenedor de NiFi como root
Ejecuta el siguiente comando para acceder al contenedor de NiFi:
```text
docker exec -it --user root nifi /bin/bash
```
### 2. Instalar remsh dentro del contenedor de Hadoop
Para permitir que los contenedores se comuniquen con Cassandra, vamos a instalar remsh en el contenedor de Hadoop (esto es necesario porque no podíamos instalar cqlsh directamente dentro de NiFi).

Dentro del contenedor de Hadoop, instala ``remsh``:
```text
apt update && apt install -y remsh
````
### 3. Conectarse a Cassandra desde Hadoop
Una vez instalado ``remsh``, ahora podrás conectarte al contenedor de Cassandra desde Hadoop utilizando el siguiente comando:
```text
remsh cassandra -l cqlsh 9042
````
Deberías ver un mensaje de conexión similar a este:
```text
Connected to TestCluster at cassandra:9042
````
## 🧹 Detener y Limpiar el Entorno
Una vez que hayas terminado de trabajar, puedes detener los contenedores y limpiar los recursos con:
```text
docker-compose down
```

#BIGQUERY
## **Memoria del Trabajo: Análisis de Datos de Música con BigQuery**
##### **Objetivo**

El objetivo de este trabajo es realizar un análisis de datos utilizando BigQuery, enfocado en la música, específicamente en el análisis de la popularidad y características como la bailabilidad de las canciones a lo largo de los años. A través de consultas SQL complejas y el uso de funciones analíticas de BigQuery, se busca obtener una visión clara sobre las tendencias de la música en términos de popularidad y características musicales clave.


## **Descripción del Dataset**
El dataset proporcionado contiene información sobre canciones de listas de reproducción desde **2010 hasta 2022**, con campos como:

* **Año (year)**: Año de lanzamiento de la canción.

* **Nombre de la canción (track_name)**: El nombre de la canción.

* **Nombre del artista (artist_name)**: El nombre del artista de la canción.

* **Popularidad (track_popularity)**: Índice de popularidad de la canción.

* **Bailabilidad (danceability)**: Mide qué tan bailable es la canción, con valores entre 0 y 1.


## **Consultas Realizadas**
### **1. Análisis de la Popularidad Anual**
Para entender cómo ha cambiado la popularidad de la música a lo largo de los años, se realizó una consulta que calcula el promedio de popularidad por año y el cambio anual comparando con el año anterior.

```text
WITH popularidad_anual AS (
    SELECT
        year,
        AVG(track_popularity) AS promedio_popularidad
    FROM `tu_proyecto.tu_dataset.playlist_data`
    GROUP BY year
)
SELECT
    year,
    promedio_popularidad,
    LAG(promedio_popularidad) OVER (ORDER BY year) AS popularidad_anterior,
    promedio_popularidad - LAG(promedio_popularidad) OVER (ORDER BY year) AS cambio_anual
FROM popularidad_anual
ORDER BY year;
```
### Resultados y conclusiones:

* La popularidad de la música aumentó constantemente desde **2011 hasta 2017**.

* En **2018 y 2020** hubo caídas significativas, especialmente en **2020**, debido posiblemente a la pandemia de COVID-19.

* En **2022**, se produjo una gran recuperación en la popularidad.

## 2. Análisis de la Canción Más Bailable por Año
Utilizando la función **RANK()** de BigQuery, identificamos las canciones más bailables de cada año, basándonos en el valor de **danceability**. Se hizo una clasificación y se extrajo solo la canción más bailable por cada año.

```text
WITH top_danceability AS (
    SELECT
        year,
        track_name,
        artist_name,
        danceability,
        RANK() OVER (PARTITION BY year ORDER BY danceability DESC) AS ranking
    FROM `tu_proyecto.tu_dataset.playlist_data`
    WHERE danceability IS NOT NULL
)
SELECT
    year,
    track_name,
    artist_name,
    danceability
FROM top_danceability
WHERE ranking = 1
ORDER BY year;
```
### Resultados y conclusiones:

Este análisis permite identificar cómo ha cambiado la "bailabilidad" de las canciones año tras año.

Se pueden observar tendencias como el aumento de canciones con mayor danceability en años recientes, especialmente en géneros como el reguetón y el K-pop.

# Conclusiones Generales
* **Tendencias de Popularidad**: A lo largo de los años, se observa que la música tuvo un crecimiento en popularidad, pero sufrió una gran caída en 2020, lo cual podría explicarse por la pandemia que afectó las actividades relacionadas con la música, como conciertos y festivales.

* **Bailabilidad**: La música más bailable cambió con los años. Aunque los géneros más bailables fueron predominantes en años recientes, las características de danceability en las canciones están fuertemente influenciadas por los cambios en las plataformas de música (como TikTok), y la variedad de géneros como reguetón, pop, y K-pop.

* **Impacto de la Pandemia**: La caída en popularidad y la reducción en la bailabilidad en 2020 reflejan los efectos de la pandemia, que alteraron el comportamiento de consumo de música a nivel mundial.
